# Memory caching

In [Workflow notebook](basic_worflow.ipynb) you larned about ``Workflows`` that specify processing by an execution graph and offer efficient recomputing. However, sometimes you might want to use ``Interfaces`` that give better control of the execution of each step and can be easily combine with any Python code. Unfortunately, ``Interfaces`` do not offer any caching and you always dully recompute your task. 

Solution to this problem can be a ``caching`` mechanism supported by Nipype. Nipype caching relies on the ``Memory`` class and creates an execution context that is bound to a disk cache.
When you instantiate the class you should provide ``base_dir`` and additional subdirectory called ``nipype_mem`` will be automatically created. 

In [1]:
from nipype.caching import Memory
mem = Memory(base_dir='/output/workingdir')

If we want to ask for caching for the ``BET`` interface, we can use ``cache`` method that takes interfaces classes as an argument.

In [3]:
from nipype.interfaces import fsl
bet_mem = mem.cache(fsl.BET)

Now, ``bet_mem`` can be applied as a function with inputs of the ``BET`` interface as the function arguments. Those inputs are given as keyword arguments, bearing the same name as the name in the inputs specs of the interface.

In [4]:
bet_mem(in_file="/data/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz",
        out_file="/output/sub-02_T1w_brain.nii.gz",
        mask=True)

170809-21:22:39,27 workflow INFO:
	 Executing node 0cb4fbaf1687a0cccdf631a94c9deece in dir: /output/workingdir/nipype_mem/nipype-interfaces-fsl-preprocess-BET/0cb4fbaf1687a0cccdf631a94c9deece
170809-21:22:39,88 workflow INFO:
	 Running: bet /data/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz /output/sub-02_T1w_brain.nii.gz -m


As you can seen ``bet`` command was called as expected. We can now check the content of caching file:

In [5]:
! ls -l /output/workingdir/nipype_mem

total 4
drwxr-xr-x 3 neuro users 102 Aug  9 21:22 log.2017
-rw-r--r-- 1 neuro users  70 Aug  9 21:22 log.current
drwxr-xr-x 3 neuro users 102 Aug  9 21:22 nipype-interfaces-fsl-preprocess-BET


A special subdirectory for our interface has beeen created.

Now, let's try to run the interface again for the same subject:

In [8]:
bet_mem(in_file="/data/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz",
        out_file="/output/sub-02_T1w_brain.nii.gz",
        mask=True)

170809-21:30:58,228 workflow INFO:
	 Executing node 0cb4fbaf1687a0cccdf631a94c9deece in dir: /output/workingdir/nipype_mem/nipype-interfaces-fsl-preprocess-BET/0cb4fbaf1687a0cccdf631a94c9deece
170809-21:30:58,253 workflow INFO:


As you can see, Nipype didn't run the interface again, but collected previously calculated results.

If we're having problem and want to read crash file, we can use ``runtime.cwd``. DJ - TODO: doesn't work

In [19]:
res = bet_mem(in_file="/data/ds000114/sub-01/ses-test/anat/sub-001_ses-test_T1w.nii.gz",
    out_file="/output/test_sub-01_T1w_brain.nii.gz",
    mask=True)

TraitError: The trait 'in_file' of a BETInputSpec instance is an existing file name, but the path  '/data/ds000114/sub-01/ses-test/anat/sub-001_ses-test_T1w.nii.gz' does not exist.

In [20]:
res.runtime.cwd

NameError: name 'res' is not defined

If you created cached results that you're not going reuse, you should flush the cache using [Memory.clear_previous_runs()](http://nipy.org/nipype/0.10.0/users/caching_tutorial.html#nipype.caching.Memory.clear_previous_runs) or [Memory.clear_runs_since()](http://nipy.org/nipype/0.10.0/users/caching_tutorial.html#nipype.caching.Memory.clear_runs_since).
DJ TODO: don't know how to use it - output is still collected from the cashing directory.